update workflow3_fullaccess.py (xử lý với việc folder, file ko public mà cần quyền) với: 
```bash
SCOPES = [
    'https://www.googleapis.com/auth/drive',  # Full access
    'https://www.googleapis.com/auth/drive.file',  # Access to files created/opened by the app
    'https://www.googleapis.com/auth/drive.readonly'  # Read-only access to files
]
```
và 
Thay thế gdown.download bằng phương thức download_file mới sử dụng Google Drive API trực tiếp

# 1. Update full quyền 

Tôi thấy vấn đề nằm ở Config của bạn. Trong code hiện tại, SCOPES chỉ có quyền read metadata:

```python
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
```

Nhưng bên dưới lại có một SCOPES khác với full quyền nhưng không được sử dụng:

```python
# Update SCOPES to include full drive access  
SCOPES = [
    'https://www.googleapis.com/auth/drive',  # Full access
    'https://www.googleapis.com/auth/drive.file',  # Access to files created/opened by the app
    'https://www.googleapis.com/auth/drive.readonly'  # Read-only access to files
]
```

Hãy sửa lại Config class như sau:

```python
class Config:
    """Configuration settings"""
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    TRANSCRIBE_API_URL = 'http://103.253.20.13:25029/role_assign'
    ANALYZE_API_URL = 'http://localhost:25034/analyze'
    SECRET_KEY = 'codedongian'
    
    # Cập nhật SCOPES với đầy đủ quyền
    SCOPES = [
        'https://www.googleapis.com/auth/drive',  # Full access
        'https://www.googleapis.com/auth/drive.file',  # Access to files created/opened by the app
        'https://www.googleapis.com/auth/drive.metadata.readonly'  # Read metadata
    ]
    
    CREDENTIALS_PATH = os.path.join(BASE_DIR, 'credentials.json')
    TOKEN_PATH = os.path.join(BASE_DIR, 'token.pickle')
    DATABASE_PATH = os.path.join(BASE_DIR, 'video_database.db')
    TEMP_DIR = os.path.join(BASE_DIR, 'temp_files')
    LOG_DIR = os.path.join(BASE_DIR, 'logs')

```

Và trong VideoProcessor, thêm parameters cho list API:

```python
class VideoProcessor:
    # ... other methods ...

    def find_folders(self, folder_id: str) -> Tuple[Optional[str], Optional[str]]:
        self.logger.info(f"Looking for Audio and Video folders in {folder_id}")
        query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder'"
        results = self.drive_service.files().list(
            q=query, 
            fields="files(id, name)",
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()
        
        audio_folder = None
        video_folder = None
        
        files = results.get('files', [])
        self.logger.info(f"Found {len(files)} folders:")
        for item in files:
            self.logger.info(f"- Found folder: {item['name']}")
            if item['name'] == 'Audio':
                audio_folder = item['id']
                self.logger.info(f"Found Audio folder: {audio_folder}")
            elif item['name'] == 'Video':
                video_folder = item['id']
                self.logger.info(f"Found Video folder: {video_folder}")
        
        return audio_folder, video_folder

    def process_folder(self, folder_id: str):
        try:
            # Verify folder access first
            try:
                folder = self.drive_service.files().get(
                    fileId=folder_id,
                    supportsAllDrives=True
                ).execute()
                self.logger.info(f"Successfully accessed folder: {folder.get('name', 'Unknown')}")
            except Exception as e:
                self.logger.error(f"Cannot access folder: {str(e)}")
                return

            # Find folders
            audio_folder, video_folder = self.find_folders(folder_id)
            if not audio_folder or not video_folder:
                self.logger.error("Could not find both Audio and Video folders")
                return

            # Get list of videos
            self.logger.info(f"Getting videos from folder: {video_folder}")
            videos = self.drive_service.files().list(
                q=f"'{video_folder}' in parents",
                fields="files(id, name)",
                supportsAllDrives=True,
                includeItemsFromAllDrives=True
            ).execute().get('files', [])
            self.logger.info(f"Found {len(videos)} videos")

            # Rest of the code remains the same...

```

Các thay đổi chính:

1. Sửa SCOPES trong Config để có đủ quyền truy cập
2. Thêm parameters `supportsAllDrives=True` và `includeItemsFromAllDrives=True` cho các API calls
3. Thêm logging chi tiết hơn
4. Thêm kiểm tra quyền truy cập folder trước khi xử lý

Các bước thực hiện:

1. Cập nhật code như trên
2. Xóa file token.pickle cũ
3. Chạy lại chương trình
4. Khi authorize, chọn tài khoản có quyền truy cập folder
5. Chấp nhận tất cả quyền được yêu cầu

Việc này sẽ giúp:
- Cấp đúng quyền cho ứng dụng
- Hỗ trợ truy cập shared drives
- Log chi tiết hơn để debug

# 2. Update để tìm những cái bên trong 2 folder: Video, Audio

oke tốt đó. Giờ đã tìm được 2 folder bên trong là: Audio, Video rùi nhỉ 

base) PS D:\OneDrive - Hanoi University of Science and Technology\GIT\grading-video-mentor> & "d:/OneDrive - Hanoi University of Science and Technology/GIT/grading-video-mentor/.venv/Scripts/python.exe" "d:/OneDrive - Hanoi University of Science and Technology/GIT/grading-video-mentor/deploy1.3.3_combine/backend/database/workflow3_fullquyen.py"
2024-11-06 16:39:42,944 - INFO - Database initialized
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=333804057110-vluunnk2bk7e89a4bpc7rdv0cc3m33vv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64260%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&state=ks8KfiE5mVA5YxNCImJMrOSyBr4W23&access_type=offline
2024-11-06 16:39:51,655 - INFO - Successfully accessed folder: Test
2024-11-06 16:39:51,656 - INFO - Looking for Audio and Video folders in 15ik6vWdz1IcbPmsPfr4p-DdszkB484yg   
2024-11-06 16:39:52,493 - INFO - Found 2 folders:
2024-11-06 16:39:52,494 - INFO - - Found folder: Audio
2024-11-06 16:39:52,494 - INFO - Found Audio folder: 1Ofz3dMbLnrH3Ru0iVDPHFRVFd86f1_aB
2024-11-06 16:39:52,495 - INFO - - Found folder: Video
2024-11-06 16:39:52,495 - INFO - Found Video folder: 1JC3ULemPIZ5wA7w5GX3Zvf7V9dM5FFfC
2024-11-06 16:39:52,496 - INFO - Getting videos from folder: 1JC3ULemPIZ5wA7w5GX3Zvf7V9dM5FFfC
2024-11-06 16:39:52,872 - INFO - Found 0 videos
2024-11-06 16:39:52,872 - INFO -
Processed Videos:
(base) PS D:\OneDrive - Hanoi University of Science and Technology\GIT\grading-video-mentor>


=========
Tôi nghĩ là: bạn vừa update code 
Sửa SCOPES trong Config để có đủ quyền truy cập
Thêm parameters supportsAllDrives=True và includeItemsFromAllDrives=True cho các API calls

- do đó từ folder Test mới tìm được 2 folder: Audio, Video

Giờ bạn cần update code để từ 2 folder Audio, Video -> tìm được các file bên trong nhé. 

Tôi thấy vấn đề nằm ở Config của bạn. Trong code hiện tại, SCOPES chỉ có quyền read metadata:
```python
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
```
Nhưng bên dưới lại có một SCOPES khác với full quyền nhưng không được sử dụng:

```python
# Update SCOPES to include full drive access  
SCOPES = [
    'https://www.googleapis.com/auth/drive',  # Full access
    'https://www.googleapis.com/auth/drive.file',  # Access to files created/opened by the app
    'https://www.googleapis.com/auth/drive.readonly'  # Read-only access to files
]
```

```python
def process_folder(self, folder_id: str):
    try:
        # Verify folder access first
        try:
            folder = self.drive_service.files().get(
                fileId=folder_id,
                supportsAllDrives=True
            ).execute()
            self.logger.info(f"Successfully accessed folder: {folder.get('name', 'Unknown')}")
        except Exception as e:
            self.logger.error(f"Cannot access folder: {str(e)}")
            return

        # Find Audio and Video folders
        audio_folder, video_folder = self.find_folders(folder_id)
        if not audio_folder or not video_folder:
            self.logger.error("Could not find both Audio and Video folders")
            return

        # List files in Video folder
        self.logger.info(f"\nListing files in Video folder:")
        videos = self.drive_service.files().list(
            q=f"'{video_folder}' in parents",
            fields="files(id, name, mimeType)",
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute().get('files', [])
        
        self.logger.info(f"Found {len(videos)} videos:")
        for video in videos:
            self.logger.info(f"- Video: {video['name']} (ID: {video['id']})")

        # List files in Audio folder
        self.logger.info(f"\nListing files in Audio folder:")
        audio_files = self.drive_service.files().list(
            q=f"'{audio_folder}' in parents",
            fields="files(id, name, mimeType)",
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute().get('files', [])
        
        self.logger.info(f"Found {len(audio_files)} audio files:")
        for audio in audio_files:
            self.logger.info(f"- Audio: {audio['name']} (ID: {audio['id']})")

        # Process each video
        for video in videos:
            self.logger.info(f"\nProcessing video: {video['name']}")
            video_url = f"https://drive.google.com/file/d/{video['id']}/view"
            
            # Check if already processed
            if self.db.url_exists(video_url):
                self.logger.info(f"Skipping existing video: {video['name']}")
                continue

            # Find corresponding audio files
            self.logger.info(f"Looking for audio files for video: {video['name']}")
            base_name = os.path.splitext(video['name'])[0]
            
            # List all matching audio files
            matching_audio = [
                f for f in audio_files 
                if base_name in f['name']
            ]
            
            self.logger.info(f"Found {len(matching_audio)} related audio files:")
            for audio in matching_audio:
                self.logger.info(f"- Related audio: {audio['name']}")

            # Find HV and mentor files
            hv_file = None
            mentor_file = None
            for file in matching_audio:
                self.logger.info(f"Checking audio file: {file['name']}")
                if 'HV' in file['name']:
                    hv_file = file
                    self.logger.info(f"Found HV file: {file['name']} (ID: {file['id']})")
                elif 'mentor' in file['name']:
                    mentor_file = file
                    self.logger.info(f"Found mentor file: {file['name']} (ID: {file['id']})")

            if not (hv_file and mentor_file):
                self.logger.error(f"Missing audio files for video: {video['name']}")
                continue

            try:
                # Download and process HV file
                self.logger.info(f"Processing HV file: {hv_file['name']}")
                hv_path = os.path.join(self.config.TEMP_DIR, f"hv_{video['id']}.wav")
                self.logger.info(f"Downloading HV file to: {hv_path}")
                gdown.download(
                    f"https://drive.google.com/uc?id={hv_file['id']}", 
                    hv_path, 
                    quiet=False,
                    use_cookies=False  # Add this if gdown has issues
                )
                hv_result = self.audio_processor.process_audio(hv_path)

                # Download and process mentor file
                self.logger.info(f"Processing mentor file: {mentor_file['name']}")
                mentor_path = os.path.join(self.config.TEMP_DIR, f"mentor_{video['id']}.wav")
                self.logger.info(f"Downloading mentor file to: {mentor_path}")
                gdown.download(
                    f"https://drive.google.com/uc?id={mentor_file['id']}", 
                    mentor_path, 
                    quiet=False,
                    use_cookies=False  # Add this if gdown has issues
                )
                mentor_result = self.audio_processor.process_audio(mentor_path)

                if hv_result and mentor_result:
                    self.logger.info("Combining transcriptions...")
                    transcript = self.audio_processor.combine_transcriptions(
                        hv_result, mentor_result)
                    
                    self.logger.info("Getting criteria...")
                    criteria = self.audio_processor.analyze_transcript(transcript)

                    # Save to database
                    self.logger.info("Saving to database...")
                    if self.db.insert_video(video_url, transcript, criteria):
                        self.logger.info(f"Successfully processed video: {video['name']}")

                # Cleanup
                self.logger.info("Cleaning up temporary files...")
                for path in [hv_path, mentor_path]:
                    if os.path.exists(path):
                        os.remove(path)
                        self.logger.info(f"Removed temporary file: {path}")

            except Exception as e:
                self.logger.error(f"Error processing video {video['name']}: {str(e)}")
                continue

            time.sleep(3)  # Add delay between processing videos

    except Exception as e:
        self.logger.error(f"Error in main processing: {str(e)}")
```

# 3. Quyền down file 

Lỗi này xảy ra vì gdown không thể tải file từ Google Drive do vấn đề về quyền truy cập. Chúng ta cần thay đổi cách tải file bằng cách sử dụng Google Drive API trực tiếp thay vì gdown. Đây là cách sửa:

```python:deploy1.3.3_combine/backend/database/workflow3_fullquyen.py
class VideoProcessor:
    # ... existing methods ...

    def download_file(self, file_id: str, destination: str) -> bool:
        """Download a file from Google Drive using Drive API"""
        try:
            request = self.drive_service.files().get_media(
                fileId=file_id,
                supportsAllDrives=True
            )
            
            with open(destination, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    if status:
                        self.logger.info(f"Download {int(status.progress() * 100)}%")
            return True
        except Exception as e:
            self.logger.error(f"Error downloading file: {str(e)}")
            return False

    def process_folder(self, folder_id: str):
        """Process a shared folder"""
        try:
            # ... existing code until audio file processing ...

            for video in videos:
                # ... existing code until downloading files ...

                try:
                    # Download and process HV file
                    self.logger.info(f"Processing HV file: {hv_file['name']}")
                    hv_path = os.path.join(self.config.TEMP_DIR, f"hv_{video['id']}.wav")
                    self.logger.info(f"Downloading HV file to: {hv_path}")
                    
                    # Use new download method instead of gdown
                    if not self.download_file(hv_file['id'], hv_path):
                        raise Exception(f"Failed to download HV file: {hv_file['name']}")
                    
                    hv_result = self.audio_processor.process_audio(hv_path)

                    # Download and process mentor file
                    self.logger.info(f"Processing mentor file: {mentor_file['name']}")
                    mentor_path = os.path.join(self.config.TEMP_DIR, f"mentor_{video['id']}.wav")
                    self.logger.info(f"Downloading mentor file to: {mentor_path}")
                    
                    # Use new download method instead of gdown
                    if not self.download_file(mentor_file['id'], mentor_path):
                        raise Exception(f"Failed to download mentor file: {mentor_file['name']}")
                    
                    mentor_result = self.audio_processor.process_audio(mentor_path)

                    # ... rest of the processing code ...

                except Exception as e:
                    self.logger.error(f"Error processing video {video['name']}: {str(e)}")
                    continue

                # ... rest of the code ...

```

Đồng thời, thêm import cần thiết ở đầu file:

```python:deploy1.3.3_combine/backend/database/workflow3_fullquyen.py
from googleapiclient.http import MediaIoBaseDownload
```

### Những thay đổi chính:

1. Thêm phương thức `download_file` mới sử dụng Google Drive API trực tiếp
2. Thay thế gdown.download bằng phương thức download_file mới
3. Thêm xử lý lỗi chi tiết hơn
4. Sử dụng supportsAllDrives=True để hỗ trợ Shared Drive

Phương pháp này sẽ:
- Sử dụng xác thực OAuth đã có
- Hoạt động với các file trong Shared Drive
- Không bị giới hạn bởi quyền truy cập công khai
- Có thể theo dõi tiến trình tải xuống

Hãy thử chạy lại code sau khi áp dụng những thay đổi này. Nó sẽ giải quyết vấn đề không thể tải file từ Google Drive.

# 4. Về luồng 

Sau khi tìm và tải 2 file audio về, quy trình xử lý sẽ như sau:

1. Xử lý audio để lấy transcription
2. Kết hợp transcription từ cả 2 file
3. Phân tích transcription để lấy criteria
4. Lưu kết quả vào database

Đây là code chi tiết cho quy trình:

```python:deploy1.3.3_combine/backend/database/workflow3_fullquyen.py
def process_folder(self, folder_id: str):
    try:
        # ... existing code for finding and downloading files ...

        for video in videos:
            try:
                # ... code for finding and downloading audio files ...
                
                # 1. Process both audio files to get transcriptions
                self.logger.info("Processing audio files for transcription...")
                
                # Process HV audio
                hv_result = self.audio_processor.process_audio(hv_path)
                if not hv_result:
                    raise Exception("Failed to process HV audio file")
                self.logger.info("Successfully processed HV audio")
                
                # Process mentor audio
                mentor_result = self.audio_processor.process_audio(mentor_path)
                if not mentor_result:
                    raise Exception("Failed to process mentor audio file")
                self.logger.info("Successfully processed mentor audio")

                # 2. Combine transcriptions
                self.logger.info("Combining transcriptions...")
                transcript = self.audio_processor.combine_transcriptions(
                    hv_result, 
                    mentor_result
                )
                if not transcript:
                    raise Exception("Failed to combine transcriptions")
                self.logger.info("Successfully combined transcriptions")
                
                # Log sample of combined transcript
                self.logger.info(f"Combined transcript sample: {transcript[:200]}...")

                # 3. Analyze combined transcript
                self.logger.info("Analyzing transcript...")
                criteria = self.audio_processor.analyze_transcript(transcript)
                if not criteria:
                    raise Exception("Failed to analyze transcript")
                self.logger.info("Successfully analyzed transcript")
                
                # Log sample of criteria
                self.logger.info(f"Criteria sample: {criteria[:200]}...")

                # 4. Save to database
                video_url = f"https://drive.google.com/file/d/{video['id']}/view"
                self.logger.info(f"Saving results to database for video: {video['name']}")
                
                if self.db.insert_video(video_url, transcript, criteria):
                    self.logger.info(f"Successfully processed and saved video: {video['name']}")
                else:
                    raise Exception("Failed to save to database")

                # Cleanup temporary files
                self.logger.info("Cleaning up temporary files...")
                for path in [hv_path, mentor_path]:
                    if os.path.exists(path):
                        os.remove(path)
                        self.logger.info(f"Removed temporary file: {path}")

            except Exception as e:
                self.logger.error(f"Error processing video {video['name']}: {str(e)}")
                # Clean up any temporary files if they exist
                for path in [hv_path, mentor_path]:
                    if os.path.exists(path):
                        os.remove(path)
                continue

            # Add a small delay between processing videos
            time.sleep(3)

    except Exception as e:
        self.logger.error(f"Error in main processing: {str(e)}")
```

### Giải thích chi tiết quy trình:

1. **Xử lý Audio Files**:
   - Gửi từng file audio đến API transcribe
   - API sẽ trả về text từ audio
   - Kiểm tra kết quả từ cả 2 file

2. **Kết hợp Transcriptions**:
   - Lấy text từ cả file HV và mentor
   - Kết hợp thành một transcript hoàn chỉnh
   - Sắp xếp theo thời gian
   - Format với nhãn "Mentee" và "Mentor"

3. **Phân tích Transcript**:
   - Gửi transcript đã kết hợp đến API analyze
   - API sẽ phân tích và trả về criteria
   - Kiểm tra kết quả phân tích

4. **Lưu trữ Kết quả**:
   - Lưu URL video, transcript và criteria vào database
   - Xử lý lỗi nếu có
   - Dọn dẹp file tạm

5. **Xử lý Lỗi và Cleanup**:
   - Log mọi bước xử lý
   - Xử lý các trường hợp lỗi
   - Đảm bảo dọn dẹp file tạm ngay cả khi có lỗi

Mỗi bước đều có logging chi tiết để dễ dàng debug nếu có vấn đề xảy ra. Nếu bất kỳ bước nào thất bại, quy trình sẽ skip video đó và chuyển sang video tiếp theo.

Theo code hiện tại, mỗi video được xử lý độc lập và các file tạm thời sẽ được xóa ngay sau khi xử lý xong video đó, không cần đợi tất cả video xử lý xong. Tuy nhiên, chúng ta nên thêm một số kiểm tra để đảm bảo việc này hoạt động chính xác. Đây là code được cải tiến: